In [1]:
import jupyter_utils as ju

# Loading model

In [6]:
model = ju.load_model("Experiment", "250709_iJN1463_updated_5.sbml")

In [86]:
model.reactions.get_by_id('SK_adpac_c').bounds = -3.3350, 0.0
model.reactions.get_by_id('SK_14btdl_c').bounds = -1.6675, 0.0
model.reactions.get_by_id('SK_etheglycol_c').bounds = -3.3175, 0.0
model.reactions.get_by_id('SK_terepa_c').bounds = -1.65, 0.0
model.reactions.get_by_id('EX_h2s_e').bounds = -0.0, 0.0

In [7]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
o2_e,EX_o2_e,13.75,0,0.00%
etheglycol_c,SK_etheglycol_c,10,2,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-10,1,50.00%
h2o_e,EX_h2o_e,-20,0,0.00%
h_e,EX_h_e,-2.5,0,0.00%
bhb_c,SK_bhb_c,-2.5,4,50.00%


added a sink reaction for BHB

In [7]:
model.add_boundary(model.metabolites.get_by_id('bhb_c'), type='sink',lb=0.0, ub=1000)

Reaction identifier,SK_bhb_c
Name,(R)-3-Hydroxybutanoate sink
Memory address,0x1994ac4cd10
Stoichiometry,bhb_c --> (R)-3-Hydroxybutanoate -->
GPR,
Lower bound,0.0
Upper bound,1000


# Calculate the carbon yield for each monomer

In [75]:
# substrate carbon number
substrate_carbons = {
    'SK_etheglycol_c': 2,
    'SK_terepa_c': 8,
    'SK_adpac_c': 6,
    'SK_14btdl_c': 4
}
model.objective='SK_bhb_c'
model.optimize()
total_sub_cflux = 0
for rxn_id in substrate_carbons:
    upt = -model.reactions.get_by_id(rxn_id).flux  # uptake positive magnitude
    print(f'{rxn_id} flux = {upt}')
    total_sub_cflux += upt * substrate_carbons[rxn_id]
prod_flux = model.reactions.get_by_id('SK_bhb_c').flux
prod_cflux = prod_flux * 4  # BHB has 4 carbons
carbon_yield = prod_cflux / total_sub_cflux if total_sub_cflux > 0 else None
print(carbon_yield)

SK_etheglycol_c flux = 3.3175
SK_terepa_c flux = 1.65
SK_adpac_c flux = 3.335
SK_14btdl_c flux = 1.6675
0.5716973019456085


In [48]:
product_flux =  model.reactions.get_by_id('SK_bhb_c').flux
substrat_flux = - model.reactions.get_by_id('SK_adpac_c').flux


yield_flux = (product_flux*4)/(substrat_flux*6)
yield_flux

1.3289605197401144

check the summary of metabolite

In [8]:
summary = model.metabolites.get_by_id('14btdl_c').summary()
summary

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


# Reactions knocked out for correct yield

In [8]:
model.reactions.get_by_id('SUCOAS').bounds = -1000.0, 0.0 
model.reactions.get_by_id('PPC').bounds = -0.0, 1000.0 # OPEN
model.reactions.get_by_id('PC').bounds = -0.0, 1000.0 # OPEN
model.reactions.get_by_id('THRS').bounds = -0.0, 0.0 

check reaction information

In [4]:
search_word = 'PC'

for reaction in model.reactions:
    if search_word == reaction.id:
        print(f"Reaction ID: {reaction.id}")
        print(f"Name: {reaction.name}")
        print(f"Equation: {reaction.reaction}")
        print(f"Lower Bound: {reaction.lower_bound}")
        print(f"Upper Bound: {reaction.upper_bound}")
        print(reaction.annotation)
        print(f"Genes: {[gene.id for gene in reaction.genes]}")

Reaction ID: PC
Name: Pyruvate carboxylase
Equation: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
Lower Bound: 0.0
Upper Bound: 999999.0
{'sbo': 'SBO:0000176', 'bigg.reaction': 'PC', 'biocyc': 'META:PYRUVATE-CARBOXYLASE-RXN', 'ec-code': '6.4.1.1', 'kegg.reaction': 'R00344', 'metanetx.reaction': 'MNXR102391', 'rhea': ['20847', '20844', '20845', '20846'], 'sabiork': '105', 'seed.reaction': 'rxn00250'}
Genes: ['PP_5347', 'PP_5346']


check metabolite information

In [76]:
search_word = 'bhb'
# Iterate over metabolites in the model
for metabolite in model.metabolites:
    if search_word in metabolite.id:
        # Print metabolite information
        print(metabolite.id)
        print(metabolite.charge)
        print(metabolite.formula)
        print(metabolite.name)
        print(metabolite.compartment)
        print(metabolite.annotation)
        
        # Get related reactions
        related_reactions = list(metabolite.reactions)
        
        # Get genes related to the reactions
        related_genes = set()
        for reaction in related_reactions:
            related_genes.update(reaction.genes)
        
        # Print related reactions and associated genes
        print(f"Related Reactions: {', '.join(r.id for r in related_reactions)}")
        print(f"Genes Associated: {', '.join(g.id for g in related_genes)}\n")

bhb_c
-1
C4H7O3
(R)-3-Hydroxybutanoate
c
{'sbo': 'SBO:0000247', 'bigg.metabolite': 'bhb', 'biocyc': 'META:CPD-335', 'chebi': ['CHEBI:18666', 'CHEBI:10983', 'CHEBI:17066', 'CHEBI:322'], 'hmdb': 'HMDB00011', 'inchi_key': 'WHBMMWSBFZVSSR-GSVOUGTGSA-M', 'kegg.compound': 'C01089', 'lipidmaps': 'LMFA01050243', 'metanetx.chemical': 'MNXM663', 'reactome.compound': ['73911', '5696461'], 'sabiork': '2245', 'seed.compound': 'cpd00797'}
Related Reactions: RHA40tpp, BDH, SK_bhb_c, BHBtpp, HBRCH, ACSPHAC40, PHADPC40
Genes Associated: PP_4549, PP_4550, PP_4762, PP_5004, PP_3073, PP_3074, PP_0763

bhb_e
-1
C4H7O3
(R)-3-Hydroxybutanoate
e
{'sbo': 'SBO:0000247', 'bigg.metabolite': 'bhb', 'biocyc': 'META:CPD-335', 'chebi': ['CHEBI:18666', 'CHEBI:10983', 'CHEBI:17066', 'CHEBI:322'], 'hmdb': 'HMDB00011', 'inchi_key': 'WHBMMWSBFZVSSR-GSVOUGTGSA-M', 'kegg.compound': 'C01089', 'lipidmaps': 'LMFA01050243', 'metanetx.chemical': 'MNXM663', 'reactome.compound': ['73911', '5696461'], 'sabiork': '2245', 'seed.compo

In [ ]:
# Base directory path
base_dir = Path.home() / "Documents" / "PhD" / "10-19 Research" / "11 Data" / "11.09_Models"
# Correct path construction using '/' operator
file_path = base_dir /"250709_iJN1463_updated_5.sbml"
sbml_filename = file_path

#json_filename = file_path

#cobra.io.json.save_json_model(model,json_filename)
cobra.io.write_sbml_model(model, sbml_filename)

# calculating the yield for different ratio

change the formula of terephtalic acid

In [13]:
metabolite_id = 'terepa_c'
metabolite = model.metabolites.get_by_id(metabolite_id)
initial_formula = metabolite.formula
metabolite.formula = 'C8H6O4'

metabolite_id = 'terepa_c'
if metabolite_id in model.metabolites:
    metabolite = model.metabolites.get_by_id(metabolite_id)
    metabolite.charge = -0
    metabolite.compartment = 'c'
    
    print(list(metabolite.reactions))  

[<Reaction SK_terepa_c at 0x19152427090>, <Reaction TEREDEOXY at 0x1915241e350>]


create the polymers metabolite in the model

In [4]:
from cobra import Model, Reaction, Metabolite, Gene
new_metabolite = Metabolite(
    id='PET_c',     # Metabolite ID
    name='PET',     # Metabolite Name
    formula='C10H8O4',         # Chemical Formula
    compartment='c'            # Compartment (e.g., cytoplasm)
)
new_metabolite.charge = -0

model.add_metabolites([new_metabolite])

new_metabolite1 = Metabolite(
    id='PU_c',     # Metabolite ID
    name='PU',     # Metabolite Name
    formula='C18H28O8',         # Chemical Formula
    compartment='c'            # Compartment (e.g., cytoplasm)
)
new_metabolite1.charge = -0

model.add_metabolites([new_metabolite1])


new_metabolite2 = Metabolite(
    id='PBAT_c',     # Metabolite ID
    name='PBAT',     # Metabolite Name
    formula='C18H26O10',         # Chemical Formula
    compartment='c'            # Compartment (e.g., cytoplasm)
)
new_metabolite2.charge = -0

model.add_metabolites([new_metabolite2])


Create the polymers reaction

In [5]:
from cobra import Model, Reaction, Metabolite, Gene

reaction = Reaction('PET_mix_uptake')
reaction.name = 'PET mixture uptake'
reaction.lower_bound =  0.0
reaction.upper_bound = 1000.0
etheglycol = model.metabolites.get_by_id('etheglycol_c')
terepa = model.metabolites.get_by_id('terepa_c')
pet = model.metabolites.get_by_id('PET_c')
h2o = model.metabolites.get_by_id('h2o_c')
reaction.add_metabolites({etheglycol:-1.,terepa:-1.,pet:1.,h2o:2.})
model.add_reactions([reaction])

reaction1 = Reaction('PU_mix_uptake')
reaction1.name = 'PU mixture uptake'
reaction1.lower_bound =  0.0
reaction1.upper_bound = 1000.0
etheglycol = model.metabolites.get_by_id('etheglycol_c')
adpac = model.metabolites.get_by_id('adpac_c')
btdl = model.metabolites.get_by_id('14btdl_c')
pu = model.metabolites.get_by_id('PU_c')
h2o = model.metabolites.get_by_id('h2o_c')
reaction1.add_metabolites({etheglycol:-1.0,adpac:-2.,btdl:-1.,pu:1.,h2o:4.})
model.add_reactions([reaction1])

reaction2 = Reaction('PBAT_mix_uptake')
reaction2.name = 'PBAT mixture uptake'
reaction2.lower_bound =  0.0
reaction2.upper_bound = 1000.0
terepa = model.metabolites.get_by_id('terepa_c')
adpac = model.metabolites.get_by_id('adpac_c')
btdl = model.metabolites.get_by_id('14btdl_c')
pbat = model.metabolites.get_by_id('PBAT_c')
h2o = model.metabolites.get_by_id('h2o_c')
reaction2.add_metabolites({terepa:-0.88,adpac:-1.112,btdl:-2.,pbat:1.})
model.add_reactions([reaction2])


Ratio variation

In [106]:
import numpy as np

# polymer composition dictionaries
mixture_PU = {
    'SK_etheglycol_c': 0.25,
    'SK_terepa_c': 0.0,
    'SK_adpac_c': 0.5,
    'SK_14btdl_c': 0.25
}

mixture_PET = {
    'SK_etheglycol_c': 0.5,
    'SK_terepa_c': 0.5,
    'SK_adpac_c': 0.0,
    'SK_14btdl_c': 0.0
}

mixture_PBAT = {
    'SK_etheglycol_c': 0.0,
    'SK_terepa_c': 0.222,
    'SK_adpac_c': 0.278,
    'SK_14btdl_c': 0.5
}

substrate_carbons = {
    'SK_etheglycol_c': 2,
    'SK_terepa_c': 8,
    'SK_adpac_c': 6,
    'SK_14btdl_c': 4
}

polymers = {
    "PET": mixture_PET,
    "PU": mixture_PU,
    "PBAT": mixture_PBAT
}

total_uptake = 10.0  # mmol/gDW/h
# results storage
rows = []

# --------- LOOP OVER 3 BATCHES ---------
for fixed_polymer in ["PET", "PU", "PBAT"]:
    print(f"\n--- Batch with {fixed_polymer} fixed at 0.33 ---")

    for i in np.arange(0.0, 0.5, 0.01):  # the remaining 0.667 split between other 2
        # reset flux tracking
        total_sub_cflux = 0.0

        # assign fractions
        fractions = {"PET": 0.0, "PU": 0.0, "PBAT": 0.0}
        fractions[fixed_polymer] = 0.5

        others = [p for p in polymers.keys() if p != fixed_polymer]
        fractions[others[0]] = i
        fractions[others[1]] = 0.5 - i

        # ----  SUM TOTAL UPTAKE PER MET ----
        uptake_dict = {met: 0.0 for met in substrate_carbons}
        for polymer, mixture in polymers.items():
            for met, coeff in mixture.items():
                uptake_dict[met] += total_uptake * fractions[polymer] * coeff

        # apply once per metabolite
        for met, lb_value in uptake_dict.items():
            rxn = model.reactions.get_by_id(met)
            rxn.bounds = (-lb_value, 0.0)

        print(f"Fractions → PET={fractions['PET']}, PU={fractions['PU']}, PBAT={fractions['PBAT']}")

        # optimize model
        model.objective = "SK_bhb_c"
        sol = model.optimize()

        # carbon balance
        for rxn_id in substrate_carbons:
            upt = -model.reactions.get_by_id(rxn_id).flux
            print(f'{rxn_id} flux = {upt}')
            total_sub_cflux += upt * substrate_carbons[rxn_id]

        prod_flux = sol.fluxes["SK_bhb_c"]
        prod_cflux = prod_flux * 4  # BHB has 4 carbons
        carbon_yield = prod_cflux / total_sub_cflux if total_sub_cflux > 0 else None

        print(f"Carbon yield = {carbon_yield}")

        
        rows.append({
            "fixed_polymer": fixed_polymer,
            "var_frac": i,
            "frac_PET": fractions["PET"],
            "frac_PU": fractions["PU"],
            "frac_PBAT": fractions["PBAT"],
            "carbon_yield": carbon_yield,
            "prod_flux": prod_flux,
        })

# convert to DataFrame
df_ratios_third = pd.DataFrame(rows)
# save to CSV if you like:
df_ratios_third.to_excel("250924_polymer_mixture_yield_sweep_half.xlsx", index=False)


--- Batch with PET fixed at 0.33 ---
Fractions → PET=0.5, PU=0.0, PBAT=0.5
SK_etheglycol_c flux = 2.5
SK_terepa_c flux = 3.6100000000000003
SK_adpac_c flux = 1.3900000000000001
SK_14btdl_c flux = 2.5
Carbon yield = 0.5266181539639989
Fractions → PET=0.5, PU=0.01, PBAT=0.49
SK_etheglycol_c flux = 2.525
SK_terepa_c flux = 3.5878
SK_adpac_c flux = 1.4122000000000003
SK_14btdl_c flux = 2.475
Carbon yield = 0.5270922540939578
Fractions → PET=0.5, PU=0.02, PBAT=0.48
SK_etheglycol_c flux = 2.55
SK_terepa_c flux = 3.5656
SK_adpac_c flux = 1.4344000000000001
SK_14btdl_c flux = 2.4499999999999997
Carbon yield = 0.5275680745398922
Fractions → PET=0.5, PU=0.03, PBAT=0.47
SK_etheglycol_c flux = 2.575
SK_terepa_c flux = 3.5434
SK_adpac_c flux = 1.4566
SK_14btdl_c flux = 2.425
Carbon yield = 0.5280456246823071
Fractions → PET=0.5, PU=0.04, PBAT=0.46
SK_etheglycol_c flux = 2.6
SK_terepa_c flux = 3.5212000000000003
SK_adpac_c flux = 1.4788000000000001
SK_14btdl_c flux = 2.4000000000000004
Carbon yield

need to correct the range of the variying fraction it doesnt reach the full fraction

In [10]:
import numpy as np
import pandas as pd
from cobra import Model  # only for type hints, you probably already imported your model

# TODO : CALCULATE MOL/MOL maybe flux divide it by flux

# =======================
# --- USER PARAMETERS ---
# =======================

# Composition of each polymer (fraction of each monomer)
POLYMERS = {
    "PET":  {"SK_etheglycol_c": 0.5, "SK_terepa_c": 0.5, "SK_adpac_c": 0.00, "SK_14btdl_c": 0.00},
    "PU":   {"SK_etheglycol_c": 0.25, "SK_terepa_c": 0.00, "SK_adpac_c": 0.5, "SK_14btdl_c": 0.25},
    "PBAT": {"SK_etheglycol_c": 0.00, "SK_terepa_c": 0.222,"SK_adpac_c": 0.278,"SK_14btdl_c": 0.5},
}

# Number of carbons per monomer
CARBONS = {
    'SK_etheglycol_c': 2,
    'SK_terepa_c': 8,
    'SK_adpac_c': 6,
    'SK_14btdl_c': 4
}

TOTAL_UPTAKE = 10.0       # mmol/gDW/h total uptake flux
PRODUCT_ID = "SK_bhb_c"   # product to maximize
PRODUCT_CARBONS = 4       # BHB has 4 carbons
fixed_fraction_value = 0.5 # value of the fixed polymer


# ==========================
# --- HELPER FUNCTIONS ---
# ==========================

def set_polymer_uptake(model, fractions, total_uptake=TOTAL_UPTAKE):
    """
    Apply uptake bounds for each monomer according to polymer fractions.
    fractions: dict with polymer names and their fraction of total uptake
    """
    # Reset uptake for each substrate
    uptake_dict = {met: 0.0 for met in CARBONS}
    
    # Sum contribution from each polymer
    for polymer, mix in POLYMERS.items():
        for met, coeff in mix.items():
            uptake_dict[met] += total_uptake * fractions.get(polymer, 0) * coeff
    
    # Apply to model (set lower bounds negative)
    for met, lb in uptake_dict.items():
        if lb != 0:
            rxn = model.reactions.get_by_id(met)
            rxn.bounds = (-lb, 0.0)
    return uptake_dict


def compute_carbon_yield(model, uptake_dict, product_id=PRODUCT_ID, product_carbons=PRODUCT_CARBONS):
    """
    Optimize the model for the chosen product and compute carbon yield.
    
    """
    model.objective = product_id
    sol = model.optimize()
    
    # Carbon in consumed substrates
    total_C_in = sum((-model.reactions.get_by_id(m).flux) * c
                     for m, c in CARBONS.items())
    
    # Carbon in product
    prod_flux = sol.fluxes.get(product_id, 0)
    total_C_out = prod_flux * product_carbons
    
    yield_C = total_C_out / total_C_in if total_C_in > 0 else None
    return yield_C, prod_flux

def compute_mol_yield(model, uptake_dict, product_id=PRODUCT_ID, product_carbons=PRODUCT_CARBONS):
    """
    Optimize the model for the chosen product and compute mol/mol yield.
    
    """
    model.objective = product_id
    sol = model.optimize()
    
    total_mol_in = sum((-model.reactions.get_by_id(m).flux) 
                     for m, c in CARBONS.items())
    # Carbon in product
    prod_flux = sol.fluxes.get(product_id, 0)
    total_C_out = prod_flux * product_carbons
    
    yield_mol = total_C_out / total_mol_in if total_mol_in > 0 else None
    return yield_mol, prod_flux


def sweep_polymer_ratios(model, fixed_polymer, fixed_fraction=fixed_fraction_value,
                         total_uptake=TOTAL_UPTAKE, step=0.01):
    """
    Sweep combinations where one polymer is fixed and the other two vary.
    Returns a list of dicts with results.
    """
    rows = []
    others = [p for p in POLYMERS if p != fixed_polymer]

    for var_frac in np.arange(0, fixed_fraction_value + step/2, step):  # remaining half
        fracs = {fixed_polymer: fixed_fraction}
        fracs[others[0]] = var_frac
        fracs[others[1]] = (1-fixed_fraction) - var_frac

        # Set model uptake
        uptake_dict = set_polymer_uptake(model, fracs, total_uptake)
        
        # Optimize & get carbon yield
        yield_C, prod_flux = compute_carbon_yield(model, uptake_dict)
                
        # Optimize & get mol/mol yield
        yield_mol, prod_flux = compute_mol_yield(model, uptake_dict)
        
        rows.append({
            "fixed_polymer": fixed_polymer,
            "var_polymer": others[0],
            "var_fraction": var_frac,
            "frac_PET": fracs.get("PET", 0),
            "frac_PU": fracs.get("PU", 0),
            "frac_PBAT": fracs.get("PBAT", 0),
            "mol/mol_yield": yield_mol,
            "product_flux": prod_flux
        })
    return rows


# ===========================
# --- MAIN EXECUTION ------
# ===========================

all_results = []
for fixed in POLYMERS.keys():
    print(f"Running sweep with {fixed} fixed at {fixed_fraction_value} ...")
    all_results.extend(sweep_polymer_ratios(model, fixed_polymer=fixed))

df = pd.DataFrame(all_results)
df.to_excel("251221_polymer_mixture_yield_sweep_half_inital_constrains.xlsx", index=False)

print("\nDone! Results saved to 251218_polymer_mixture_yield_sweep_half.xlsx")

Running sweep with PET fixed at 0.5 ...
Running sweep with PU fixed at 0.5 ...
Running sweep with PBAT fixed at 0.5 ...

Done! Results saved to 251218_polymer_mixture_yield_sweep_half.xlsx
